# Import Packages

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import requests
import json
from dotenv import load_dotenv
import os
from google.cloud import bigquery, bigquery_storage
from joblib import Parallel, delayed
from ratelimiter import RateLimiter
import warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

True

# Define Your Inputs

In [2]:
# Define the start date in the format "YYYY-MM-DD"
START_DATE = "2023-01-01"

# Define the end date in the format "YYYY-MM-DD"
END_DATE = "2024-09-30"

# If this is set to True, the script will add data from the last crawler run ONLY
# If it is set to False, the script will pull data from the start date to the end date.
IS_PULL_DATA_FROM_LAST_CRAWLER_RUN = False

# Define the max number of requests per minute
MAX_REQUESTS_PER_MINUTE = 170

# Define Global Non-User Defined Inputs

In [3]:
# Define the maximum tries for the backoff decorator
EXP_BACKOFF_MAX_TRIES = 10

# Define the relevant columns to be extracted from the downloaded data
RELEVANT_COLS = [
    "job_title_name", "posted_on", "company_name", "remote", "city", "plz", "salary_low",
    "salary_high", "salary_type", "job_type", "phone_number", "industry", "job_description", "crawled_timestamp", "job_source"
]

# Define the numeric columns
NUMERIC_COLS = ["plz", "salary_low", "salary_high"]

# Retrieve the Notion API token
NOTION_API_TOKEN = os.getenv("NOTION_API_TOKEN")

# Retrieve the database ID
DATABASE_ID = os.getenv("DATABASE_ID")

# Define the request headers including Authorization token
headers = {
    "Authorization": f"Bearer {NOTION_API_TOKEN}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

# Instantiate the BQ clients

In [4]:
bq_client = bigquery.Client(project="web-scraping-371310")
bq_storage_client = bigquery_storage.BigQueryReadClient()

# Define the Query

In [5]:
# Define the time period filter
if IS_PULL_DATA_FROM_LAST_CRAWLER_RUN:
    TIME_PERIOD_FILTER = "WHERE crawled_timestamp = (SELECT MAX(crawled_timestamp) FROM `web-scraping-371310.crawled_datasets.laura_indeed_data`)"
else:
    TIME_PERIOD_FILTER = f"WHERE crawled_timestamp BETWEEN TIMESTAMP('{START_DATE}') AND TIMESTAMP('{END_DATE}')"

# Create the query
raw_data_query = f"""
SELECT *
FROM `web-scraping-371310.crawled_datasets.laura_indeed_data`
{TIME_PERIOD_FILTER}
QUALIFY ROW_NUMBER() OVER (PARTITION BY job_title_name, company_name, city order by crawled_timestamp asc) = 1
"""

# Execute the query
df_indeed_data = pd.DataFrame(bq_client.query(raw_data_query).result().to_dataframe(bqstorage_client=bq_storage_client))

# Display the data frame
df_indeed_data

,job_title_name,job_type,shift_and_schedule,company_name,company_indeed_url,city,remote,salary,crawled_page_rank,job_page_url,posted_on,listing_page_url,job_description,crawled_timestamp,crawler_name,domain,salary_type,salary_low,salary_high,plz,search_query,phone_number,industry,industry_match_type,industry_match_idx
0,PhD Researcher (f/m/x) - Electrochemical conve...,,,Helmholtz-Zentrum für Umweltforschung UFZ,https://de.indeed.com/cmp/Helmholtz--zentrum-F...,Leipzig,None,None,32,https://de.indeed.com/rc/clk?jk=e23711ee9c3ad7...,Vor 13 Tagen geschaltet,https://de.indeed.com/jobs?q=Circular%20Econom...,Place of work\nLeipzig\nWorking time\n65 % (25...,2023-12-19 09:46:15+00:00,crawler_1,de,None,NaN,NaN,None,None,None,None,no_match,11772
1,Senior / Principal Water Treatment Consultant ...,Festanstellung,,Ramboll,https://de.indeed.com/cmp/Ramboll?campaignid=m...,Düsseldorf,Zum Teil im Homeoffice,None,5,https://de.indeed.com/rc/clk?jk=f619ab1748c968...,Vor 5 Tagen geschaltet,https://de.indeed.com/jobs?q=Circularity&l=Deu...,Job Description\nAny Ramboll Germany Office\nA...,2024-03-13 03:02:26+00:00,crawler_2,de,None,NaN,NaN,40476,None,None,None,no_match,11772
2,Werkstudent*in Energiemanagement (m/w/d),"Werkstudent, Befristeter Vertrag",,Zentrum für Sonnenenergie- und...,https://de.indeed.com/cmp/Zentrum-F%C3%BCr-Son...,Stuttgart,None,None,31,https://de.indeed.com/rc/clk?jk=f30394b1cb3559...,Vor > 30 Tagen geschaltet,https://de.indeed.com/jobs?q=Circular%20Econom...,// Zentrum für Sonnenenergie- und Wasserstoff-...,2024-03-13 03:09:38+00:00,crawler_1,de,None,NaN,NaN,70563,None,None,None,no_match,11772
3,Business Manager - Strategic Sustainability Co...,Festanstellung,,Ramboll,https://de.indeed.com/cmp/Ramboll?campaignid=m...,Hamburg,Zum Teil im Homeoffice,None,31,https://de.indeed.com/rc/clk?jk=4d9dd5fb66a864...,Vor > 30 Tagen geschaltet,https://de.indeed.com/jobs?q=Circular%20Econom...,"Job Description\nMünchen, Hamburg, Berlin, Fra...",2024-02-18 07:57:34+00:00,crawler_1,de,None,NaN,NaN,22763,None,None,None,no_match,11772
4,Project Manager- Automotive & Mobility Projects,,,SYSTEMIQ,https://de.indeed.com/cmp/Systemiq?campaignid=...,München,None,None,32,https://de.indeed.com/rc/clk?jk=15236c702bc05e...,Vor > 30 Tagen geschaltet,https://de.indeed.com/jobs?q=Circular%20Econom...,"Systemiq, the\nsystems change company\nand cer...",2023-12-19 09:46:19+00:00,crawler_1,de,None,NaN,NaN,None,None,None,None,no_match,11772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,"Sales Manager (M/W/D), Teil-/Vollzeit",,,SVP Deutschland AG,https://de.indeed.com/cmp/Svp-825cc399?campaig...,Hybrides Arbeiten in,None,None,63,https://de.indeed.com/rc/clk?jk=e666da1f7d0844...,Aktiv: vor 2 Tagen,https://de.indeed.com/jobs?q=Kreislaufwirtscha...,Wir suchen zum nächstmöglichen Zeitpunkt eine/...,2024-05-15 09:02:02+00:00,crawler_3,de,None,NaN,NaN,69115,None,None,None,no_match,11772
5496,Kaufmännische Sachbearbeiter (m/w/d) Back Office,,,DERICHEBOURG Umwelt GmbH,https://de.indeed.com/cmp/Derichebourg?campaig...,Weischlitz,None,None,63,https://de.indeed.com/rc/clk?jk=6b69552f2c14ec...,Aktiv: vor 23 Tagen,https://de.indeed.com/jobs?q=Kreislaufwirtscha...,Für unsere Niederlassung in Weischlitz-Reuth s...,2024-09-04 09:10:35+00:00,crawler_3,de,None,NaN,NaN,08538,None,None,None,no_match,11772
5497,Senior Frontend Developer (m/w/d),,,Mineral Minds Deutschland GmbH,https://de.indeed.com/cmp/Mineral-Minds-Deutsc...,Hybrides Arbeiten in,None,None,63,https://de.indeed.com/rc/clk?jk=2d42782b65f927...,Aktiv: vor 8 Tagen,https://de.indeed.com/jobs?q=Kreislaufwirtscha...,Für unser Entwicklungsteam suchen wir zum näch...,2024-05-08 09:03:29+00:00,crawler_3,de,None,NaN,NaN,70376,None,None,None,no_match,11772
5498,Senior Piping Engineer (m/w/d) Rohrleitungstec...,Festanstellung,,Hitachi Zosen Inova Steinmüller,https://de.indeed.com/cmp/Hitachi-Zosen-Inova-...,Gummersbach,None,None,63,https://de.indeed.com/rc/clk?jk

# Clean the data

In [6]:
# Add a new column called job_source
df_indeed_data.loc[:, "job_source"] = "Indeed Crawler"

# Add a new column called unique_identifier
df_indeed_data.loc[:, "unique_identifier"] = df_indeed_data["job_title_name"] + df_indeed_data["company_name"] + df_indeed_data["city"] + df_indeed_data["crawled_timestamp"].astype(str)

# Choose the relevant columns from the data frame
df_indeed_data = df_indeed_data[RELEVANT_COLS]

# Change the data types of numeric fields
for num_col in NUMERIC_COLS:
    df_indeed_data[num_col] = pd.to_numeric(df_indeed_data[num_col])

# Iterate over each string column and override None values with empty strings
for col in list(df_indeed_data.select_dtypes(include="object").columns):
    df_indeed_data[col].fillna("", inplace=True)

for col in list(df_indeed_data.select_dtypes(include="float64").columns):
    df_indeed_data[col].fillna(0, inplace=True)

# Display the data frame
df_indeed_data

,job_title_name,posted_on,company_name,remote,city,plz,salary_low,salary_high,salary_type,job_type,phone_number,industry,job_description,crawled_timestamp,job_source
0,PhD Researcher (f/m/x) - Electrochemical conve...,Vor 13 Tagen geschaltet,Helmholtz-Zentrum für Umweltforschung UFZ,,Leipzig,0.0,0.0,0.0,,,,,Place of work\nLeipzig\nWorking time\n65 % (25...,2023-12-19 09:46:15+00:00,Indeed Crawler
1,Senior / Principal Water Treatment Consultant ...,Vor 5 Tagen geschaltet,Ramboll,Zum Teil im Homeoffice,Düsseldorf,40476.0,0.0,0.0,,Festanstellung,,,Job Description\nAny Ramboll Germany Office\nA...,2024-03-13 03:02:26+00:00,Indeed Crawler
2,Werkstudent*in Energiemanagement (m/w/d),Vor > 30 Tagen geschaltet,Zentrum für Sonnenenergie- und...,,Stuttgart,70563.0,0.0,0.0,,"Werkstudent, Befristeter Vertrag",,,// Zentrum für Sonnenenergie- und Wasserstoff-...,2024-03-13 03:09:38+00:00,Indeed Crawler
3,Business Manager - Strategic Sustainability Co...,Vor > 30 Tagen geschaltet,Ramboll,Zum Teil im Homeoffice,Hamburg,22763.0,0.0,0.0,,Festanstellung,,,"Job Description\nMünchen, Hamburg, Berlin, Fra...",2024-02-18 07:57:34+00:00,Indeed Crawler
4,Project Manager- Automotive & Mobility Projects,Vor > 30 Tagen geschaltet,SYSTEMIQ,,München,0.0,0.0,0.0,,,,,"Systemiq, the\nsystems change company\nand cer...",2023-12-19 09:46:19+00:00,Indeed Crawler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,"Sales Manager (M/W/D), Teil-/Vollzeit",Aktiv: vor 2 Tagen,SVP Deutschland AG,,Hybrides Arbeiten in,69115.0,0.0,0.0,,,,,Wir suchen zum nächstmöglichen Zeitpunkt eine/...,2024-05-15 09:02:02+00:00,Indeed Crawler
5496,Kaufmännische Sachbearbeiter (m/w/d) Back Office,Aktiv: vor 23 Tagen,DERICHEBOURG Umwelt GmbH,,Weischlitz,8538.0,0.0,0.0,,,,,Für unsere Niederlassung in Weischlitz-Reuth s...,2024-09-04 09:10:35+00:00,Indeed Crawler
5497,Senior Frontend Developer (m/w/d),Aktiv: vor 8 Tagen,Mineral Minds Deutschland GmbH,,Hybrides Arbeiten in,70376.0,0.0,0.0,,,,,Für unser Entwicklungsteam suchen wir zum näch...,2024-05-08 09:03:29+00:00,Indeed Crawler
5498,Senior Piping Engineer (m/w/d) Rohrleitungstec...,Vor > 30 Tagen geschaltet,Hitachi Zosen Inova Steinmüller,,Gummersbach,51643.0,0.0,0.0,,Festanstellung,,,Unternehmensbeschreibung\nAls Tochtergesellsch...,2024-04-10 03:05:03+00:00,Indeed Crawler


# Create a function to return the text content of a field in the database

In [7]:
def generate_text_dict(row, col):
    """
    This function generates a dictionary with the rich text format for a given column in a row.
    """
    return {
        "rich_text": [
            {
                "text": {
                    "content": row[col]
                }
            }
        ]
    }

def generate_numeric_dict(row, col):
    """
    This function generates a dictionary with the number format for a given column in a row.
    """
    return {
        "number": row[col]
    }

# Define a function to upload data to the Notion database

In [8]:
@RateLimiter(max_calls=MAX_REQUESTS_PER_MINUTE, period=60)
def upload_row_to_notion_db(row, counter):
    """
    A function to upload a row to the Notion database
    """
    # Increment the counter so the first record is 1
    counter += 1

    # Data for the new page (record) in the database
    data = {
        "parent": {
            "database_id": DATABASE_ID
        },
        "properties": {
            # job_title_name field
            "job_title_name": {
                "title": [
                    {
                        "text": {
                            "content": row["job_title_name"]
                        }
                    }
                ]
            },

            # posted_on field
            "posted_on": generate_text_dict(row=row, col="posted_on"),
            
            # company_name field
            "company_name": generate_text_dict(row=row, col="company_name"),
            
            # remote field
            "remote": generate_text_dict(row=row, col="remote"),
            
            # city field
            "city": generate_text_dict(row=row, col="city"),
            
            # plz field
            "plz": generate_numeric_dict(row=row, col="plz"),

            # salary_low field
            "salary_low": generate_numeric_dict(row=row, col="salary_low"),

            # salary_high field
            "salary_high": generate_numeric_dict(row=row, col="salary_high"),
            
            # salary_type field
            "salary_type": generate_text_dict(row=row, col="salary_type"),

            # job_type field
            "job_type": generate_text_dict(row=row, col="job_type"),

            # phone_number field
            "phone_number": generate_text_dict(row=row, col="phone_number"),

            # industry field
            "industry": generate_text_dict(row=row, col="industry"),
            
            # job_description field
            "job_description": generate_text_dict(row=row, col="job_description"),

            # crawled_timestamp field
            "crawled_timestamp": {
                "date": {
                    "start": row["crawled_timestamp"].strftime("%Y-%m-%d %H:%M:%S")
                }
            },

            # job_source field
            "job_source": {
                "select": {
                    "name": row["job_source"]
                }
            }
        }
    }

    # Make a request to the Notion API
    response = requests.post('https://api.notion.com/v1/pages', headers=headers, data=json.dumps(data))

    # Check if the request was successful
    if response.status_code == 200:
        print(f"The job title -> {row['job_title_name']} was added successfully. This is record {counter} out of {len(df_indeed_data)}!")
    else:
        print(f"Failed to add record number {counter} with job title -> {row['job_title_name']}: ", response.status_code, response.text)

In [12]:
# Export to CSV
df_indeed_data.to_csv("indeed_data_since_the_beginning_deduplicated.csv", index=False)

# Upload the records to the Notion database

In [ ]:
parallel_job = Parallel(n_jobs=-1, backend="threading")(
    delayed(upload_row_to_notion_db)(row, counter) for counter, row in df_indeed_data.iterrows()
)

The job title -> Studentische Mitarbeiterin/ Studentischer Mitarbeiter (w/m/d) | BDI-Initiative Circular Economy was added successfully. This is record 181 out of 5500!
